# Storing Data from .csv into a PostgreSQL DB

In [1]:
import os, gzip, pandas as pd, numpy as np, matplotlib.pyplot as plt
%matplotlib inline

from sqlalchemy import create_engine, MetaData, Table, Column, Integer, String
from sqlalchemy.sql import text

from scrapy.selector import Selector

In [25]:
# Creating engine to connect to PostgreSQL
engine = create_engine([YOUR_DATABASE_HERE], echo=True)

In [72]:
source_code = './source_code'
for folder in os.listdir(source_code):
    print(folder)
    for file in os.listdir(source_code + '/' + folder):
        with gzip.open(source_code + '/' + folder + '/' + file) as f:
            file_content = f.read()
        HTML = file_content.decode()
        try:
            id = Selector(text = HTML).xpath('//*[@id="job_details"]/div[1]/div[2]/div[1]/span/text()').extract()[0].strip()
        except:
            print(file)
            continue
        #print(id)

Business Intelligence
computer vision
Data Analyst
Data Analytics
Data Engineer
Data Science
deep learning
Machine Learning
Research Scientist
Tableau
tensorflow


In [21]:
# Return dataframe based on a keyword
def keyword_to_df(keyword):
    print('Working on search query:', keyword)
    source_code = './source_code'
    features = ['job_id','job_title','emp_type','emp_seniority','job_categories','emp_min_exp','job_skills_req','job_desc','job_req','search_query', \
                'salary_freq', 'salary_low', 'salary_high', 'company_name', 'company_info', 'link']
    posting_dict = {feature:[] for feature in features}
    
    for file in os.listdir(source_code + '/' + keyword):
        with gzip.open(source_code + '/' + keyword + '/' + file) as f:
            file_content = f.read()
            HTML = file_content.decode()
            
        posting_dict['job_id'].append(Selector(text = HTML).xpath('//*[@id="job_details"]/div[1]/div[2]/div[1]/span/text()').extract()[0].strip())
        posting_dict['job_title'].append(Selector(text = HTML).xpath('//*[@id="job_title"]/text()').extract()[0].strip())
        try: posting_dict['emp_type'].append(Selector(text = HTML).xpath('//*[@id="employment_type"]/text()').extract()[0].strip())
        except: posting_dict['emp_type'].append('none')
        try: posting_dict['emp_seniority'].append(Selector(text = HTML).xpath('//*[@id="seniority"]/text()').extract()[0].strip())
        except: posting_dict['emp_seniority'].append('none')
        try: posting_dict['job_categories'].append(Selector(text = HTML).xpath('//*[@id="job-categories"]/text()').extract()[0].strip())
        except: posting_dict['job_categories'].append('none')
        try: posting_dict['emp_min_exp'].append(Selector(text = HTML).xpath('//*[@id="min_experience"]/text()').extract()[0].strip())
        except: posting_dict['emp_min_exp'].append('0 years exp')
        try: posting_dict['job_skills_req'].append(','.join(Selector(text=HTML).xpath('//*[@id="skills-needed"]/div//text()').extract()))
        except: posting_dict['job_skills_req'].append('none')
        try: posting_dict['job_desc'].append(''.join(Selector(text=HTML).xpath('//*[@id="description-content"]//text()').extract()))
        except: posting_dict['job_desc'].append('none')
        try: posting_dict['job_req'].append(''.join(Selector(text=HTML).xpath('//*[@id="requirements-content"]//text()').extract()))
        except: posting_dict['job_req'].append('none')
        posting_dict['search_query'].append(keyword)
        try: posting_dict['salary_freq'].append(Selector(text = HTML).xpath('//*[@id="job_details"]/div[1]/div[2]/div[1]/div/section[2]/div/span[3]/text()').extract()[0].strip())
        except: posting_dict['salary_freq'].append('none')
        try: posting_dict['salary_low'].append(float(Selector(text = HTML).xpath('//*[@id="job_details"]/div[1]/div[2]/div[1]/div/section[2]/div/span[2]/div/span[1]/text()').extract()[0].strip().replace('$','').replace(',','')))
        except: posting_dict['salary_low'].append(0)
        try: posting_dict['salary_high'].append(float(Selector(text = HTML).xpath('//*[@id="job_details"]/div[1]/div[2]/div[1]/div/section[2]/div/span[2]/div/span[2]/text()').extract()[0].strip().replace('$','').replace(',','')))
        except: posting_dict['salary_high'].append(0)
        try: posting_dict['company_name'].append(Selector(text = HTML).xpath('//*[@id="job_details"]/div[1]/div[2]/div[1]/section[1]/p/text()').extract()[0].strip())
        except: posting_dict['company_name'].append('none')
        try: posting_dict['company_info'].append(''.join(Selector(text=HTML).xpath('//*[@id="job_details"]/div[2]/div[1]/div/div/section/section/div[2]/div/div[1]//text()').extract()))
        except: posting_dict['company_info'].append('none')
        posting_dict['link'].append(file)
        
    return pd.DataFrame.from_dict(posting_dict)

In [22]:
test = keyword_to_df('tableau')
test.head()

Working on search query: tableau


,job_id,job_title,emp_type,emp_seniority,job_categories,emp_min_exp,job_skills_req,job_desc,job_req,search_query,salary_freq,salary_low,salary_high,company_name,company_info,link
0,JOB-2019-0084615,Account Receivable Specialist,Full Time,Junior Executive,"Accounting / Auditing / Taxation, Information ...",0 years exp,"Account Management,Customer Service,Event Plan...",What you’ll be doing… As an AR Specialist you...,Who you are… \tExperienced. Minimum associat...,tableau,Monthly,3800.0,6500.0,TABLEAU ASIA PACIFIC PTE. LTD.,Tableau Software is one of the fastest-grow...,account-receivable-specialist-tableau-asia-pac...
1,JOB-2019-0071607,"Analyst, Job",Full Time,Junior Executive,Others,0 years exp,"Change Management,Coaching,Employee Relations,...",Design and develop new data feeds and store in...,"Backgroun in IT and quantitative finance, expe...",tableau,Monthly,7000.0,7500.0,HARTREE PARTNERS SINGAPORE PTE. LIMITED,No information added.,analyst-job-hartree-partners-singapore-cbdd329...
2,JOB-2019-0087577,"Analyst (Sensing & Analytics), Ops Policy and ...",Contract,none,Public / Civil Service,0 years exp,"Army,Command,Defense,DoD,Force Protection,Gove...",Responsibilities:You will support the Ministry...,,tableau,none,0.0,0.0,Ministry of Manpower,\n\tThe Ministry of Manpower (MOM) aims to dev...,analyst-ops-policy-planning-department-1-year-...
3,JOB-2019-0073749,Analyst - Tableau (Analytics / dashboards),"Contract, Full Time",Executive,Banking and Finance,0 years exp,"Analysis,Business Analysis,Business Intelligen...",Allegis Global Solutions is the exclusive Cont...,Data Strategy team primary function is to prov...,tableau,Monthly,3000.0,6000.0,ALLEGIS GLOBAL SOLUTIONS (SINGAPORE) PTE. LTD.,Allegis Global Solutions is a full-service ...,analyst-tableau-allegis-global-solutions-d9580...
4,JOB-2019-0079550,Analytics Engineer,Permanent,Executive,Information Technology,0 years exp,"Agile Methodologies,C,C#,C++,CSS,Eclipse,Git,H...",Job Description The Analytics System team is ...,"Preferred Qualifications \tKnows Tableau, D3...",tableau,Monthly,5000.0,8000.0,ST ENGINEERING MANAGEMENT SERVICES PTE. LTD.,No information added.,analytics-engineer-st-engineering-management-s...


In [26]:
result_df = []
for i in os.listdir('./source_code'):
    df_temp = keyword_to_df(i)
    result_df.append(df_temp)
pd.concat(result_df).drop_duplicates(subset='job_id').to_sql('posting', engine, if_exists='replace', index=False)
# machine_learning = keyword_to_df('Machine Learning')
# machine_learning.drop_duplicates().to_sql('posting', engine, if_exists='replace', index=False)

Working on search query: Business Intelligence
Working on search query: computer vision
Working on search query: Data Analyst
Working on search query: Data Analytics
Working on search query: Data Engineer
Working on search query: Data Science
Working on search query: deep learning
Working on search query: Machine Learning
Working on search query: Research Scientist
Working on search query: Tableau
Working on search query: tensorflow
2019-05-05 00:47:20,713 INFO sqlalchemy.engine.base.Engine select version()
2019-05-05 00:47:20,713 INFO sqlalchemy.engine.base.Engine {}
2019-05-05 00:47:20,748 INFO sqlalchemy.engine.base.Engine select current_schema()
2019-05-05 00:47:20,752 INFO sqlalchemy.engine.base.Engine {}
2019-05-05 00:47:20,768 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2019-05-05 00:47:20,768 INFO sqlalchemy.engine.base.Engine {}
2019-05-05 00:47:20,788 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VAR

In [ ]:
# source_code = './source_code'
# features = ['id', 'company', 'title']
# posting_dict = {feature:[] for feature in features}
# for folder in ['Business Intelligence']:
#     print('Working on search query:', folder)
#     for file in os.listdir(source_code + '/' + folder):
#         with gzip.open(source_code + '/' + folder + '/' + file) as f:
#             file_content = f.read()
#         HTML = file_content.decode()
#         posting_dict['id'].append(Selector(text = HTML).xpath('//*[@id="job_details"]/div[1]/div[2]/div[1]/span/text()').extract()[0].strip())
#         posting_dict['company'].append(Selector(text = HTML).xpath('//*[@id="job_details"]/div[1]/div[2]/div[1]/section[1]/p/text()').extract()[0].strip())
#         posting_dict['title'].append(Selector(text = HTML).xpath('//*[@id="job_title"]/text()').extract()[0].strip())
# pd.DataFrame.from_dict(posting_dict).drop_duplicates().to_sql('posting', engine, if_exists='replace', index=False)

In [ ]:
# query = """
# select column_name,data_type 
# from information_schema.columns 
# where table_name = 'posting';
# """
# pd.read_sql(query, con=engine)

In [ ]:
# query = """
# ALTER TABLE posting
# ALTER COLUMN salary_low TYPE NUMERIC;
# """
# engine.execute(query)